In [ ]:
import random
import time
import sys

n = 100_000

# two lists of n random numbers from 0 to 99
x = [random.randrange(100) for _ in range(n)]
y = [random.randrange(100) for _ in range(n)]

#start counting time
start = time.time()

#make a list by multiplying the elements of the lists
i, z = 0, []
while i < n:
    z.append(x[i] * y[i])
    i += 1
    
#print elapsed time and memory
end = time.time()
mem = sys.getsizeof(z)
print('elapsed time = {}\nused memoty = {}'.format(end-start, mem))

## Alternativa: Numpy

In [ ]:
import numpy as np

x = np.array(x)
y = np.array(y)

#start counting time
start_alt = time.time()

#...
z = x * y

#print elapsed time
end_alt = time.time()
mem_alt = sys.getsizeof(z)
print('elapsed time = {}\nused memoty = {}'.format(end_alt-start_alt, mem_alt))

In [ ]:
 (end-start) - (end_alt-start_alt)

In [ ]:
mem - mem_alt

<div><img src="https://upload.wikimedia.org/wikipedia/commons/3/31/NumPy_logo_2020.svg" width=300></div>


* základní balíček pro vědecké výpočty v Pythonu
* operace mezi prvky jsou provedeny v překompilovaném C nebo Fortranu
* vektorizovaný kód
 * "zmizel" nám for cyklus
 * méně řádků = méně chyb
 
 ## ndarray
* n-rozměrné homogenní pole
* můžeme vytvořit z **listu** nebo tuplu
* vícerozměré pole je vytvořeno jako sekvence ze sekvencí

In [ ]:
arr = np.array([1,2,3])
arr

In [ ]:
arr2d = np.array([[1,2,3],[4,5,6]])
arr2d

* rozdíl mezi ndarray a std. sekvencemi:
 * homogenní
 * pevná velikost

* same data type  (dtype)

In [ ]:
arr = np.array(['s',1])
arr

In [ ]:
arr[1]+1          # ...ERROR

   *  objekt jako dtype

In [ ]:
arr = np.array(['s',[1,2]])
arr

### důležité attributy: 

* ndim

In [ ]:
arr = np.array([[1,1,1],[2,2,2]])
arr.ndim

při použití objeků...

In [ ]:
arr = np.array([[1,1],[1,2,3]])
arr.ndim

* tvar (shape)

In [ ]:
arr = np.array([[1,1,1],[2,2,2]])
arr.shape

* velikost (size)

In [ ]:
arr.size

* typ dat (dtype)

In [ ]:
arr.dtype

* veklikost prvku (itemsize)

In [ ]:
print(arr.itemsize)
sys.getsizeof(int())

* data

In [ ]:
arr.data

## Základní operace

In [ ]:
arr2d = np.array([[1,2,3,4],[8,9,10,11]])
arr2d+arr2d

In [ ]:
arr2d-arr2d

In [ ]:
arr2d*arr2d

In [ ]:
1/arr2d

In [ ]:
arr2d**2

## Broadcasting
* aneb když máme 2 pole o jiných rozměrech

In [ ]:
arr = np.arange(6)  # creates an array of integers from 0 to 6
arr.resize((2,3))  # reshapes it into a matrix
arr

In [ ]:
arr2 = np.array([[0,1,3]])
arr + arr2

* NumPy porovná tvary polí
* 2 dimenze jsou kompatibilní, pokud jsou stejné nebo jedna z nich je rovna 1

In [ ]:
# if not...
arr2 = np.array([[0,1]])
arr + arr2

## Indexování

In [ ]:
def basic_array():
    arr2d = np.zeros((6,6))
    for i in range(6):
        for j in range(6):
            arr2d[i,j] = 10*i+j
    return arr2d

arr2d = basic_array()
arr2d

In [ ]:
arr2d[0][1]

In [ ]:
arr2d[0,1]

### Indexování - Fancy Indexing
* lze použít list intů jako index
* inty nemusí být seřazené

In [ ]:
l1 = [4,2]
arr2d = basic_array()
arr2d[l1]

In [ ]:
arr2d[:,l1] = 0
arr2d

In [ ]:
# ...ERROR
l2 = [0,1,3]
arr2d[l1, l2]

Co se stalo?

In [ ]:
arr2d[[1,2,3],[1,2,3]]

Můžeme provést toto

In [ ]:
arr2d[l1][:,l2]

ale potom nefunguje přiřazování

In [ ]:
arr2d = basic_array()
arr2d[l1][:,l2] = 100
arr2d

Co se tady děje...

In [ ]:
#this notation
arr2d[l1] = 100
print(arr2d)
print("\n")
# is equal to
arr2d.__setitem__(l1, 0)
print(arr2d)

* není zapotřebí vytvořit view nebo copy (více dále)
* tzn, nevytváří se žádný nový objekt

In [ ]:
arr2d = basic_array()
# However...
arr2d[l1][:,l2] = 100
# this notation is equal to
aux = arr2d.__getitem__(l1)     # creation of new object
aux.T.__setitem__(l2, 0)
print('\naux = \n{}\n\narr2d =\n{}'.format(aux, arr2d))

## View and Copy
* view: nový pohled na ten samý úsek paměti
* copy: kopie paměti v jiné lokaci


### View

In [ ]:
arr = np.arange(5)   # base array
aux = arr.view()     # view
aux[0] = 100
arr[1] = 200
print('arr = {}'.format(arr))
print('aux = {}'.format(aux))

.base vrací None, pokud pole data vlastní, jinak vrací původní objekt

In [ ]:
aux.base

* stejný výsledek mohu získat jako

In [ ]:
arr = np.arange(5)
aux = arr
aux[0] = 100
arr[1] = 200
print('arr = {}'.format(arr))
print('aux = {}'.format(aux))

* stačí =
* co se děje?
 * pokud zavřeme obě oči (src jsem nestudoval): překlad do C++ v souboru view.cpp
* jaký je tedy smysl .view()?      ... view casting (uvedeme za chvíli)

#### View - slicing
* speciální případ View
* obdobná syntaxe jako u jiných Pytnovských objektů

In [ ]:
arr2d = basic_array()
arr2d_slice = arr2d[1:3, 3:5]    # base array
arr2d_slice[:] = 100              # view
arr2d

* nefunguje pokud kombinujeme slicing a fancy indexing

In [ ]:
arr2d_slice = arr2d[1:3, [2,3]]
arr2d_slice[:] = 77
arr2d

#### Copy

In [ ]:
arr2d = basic_array()
arr2d_slice = arr2d[1:3, 3:5].copy()
arr2d_slice[:] = 100
arr2d

In [ ]:
arr2d_slice

In [ ]:
print(arr2d_slice.base)

#### Která úprava změní původní pole?

In [ ]:
arr2d = np.array([[1,2,3],[4,5,6],[7,8,9]])
l1 = [0,2]

arr2d[:1, :][:, l1] = 100
arr2d

In [ ]:
arr2d = np.array([[1,2,3],[4,5,6],[7,8,9]])
l1 = [0,2]

arr2d[l1, :][:, :1] = 100
arr2d

## Indexování - Boolean indexing

In [ ]:
arr = np.array([1,2,3,4,5])
arr < 3

* lze použít 2 podmínky, tak jak normálně v Pythonu?

In [ ]:
arr < 3 and arr > 1

* musíme použít značení v C++ (ale NEZDVOJUJEME) + závorky

In [ ]:
(arr < 3) & (arr > 1)

In [ ]:
(arr < 3) | (arr > 3 )

In [ ]:
arr < 3 | arr > 3

* indexování

In [ ]:
arr[(arr < 3) & (arr > 1)] = 99
arr

## Subclassing
* view casting - vytvoří pohled na existující pole jako specializovanou podtřídu
* Dispatch mechanism - tvorba vlastních kontainerů polí
### view casting

In [ ]:
class C(np.ndarray):        
    def my_func(self):
        self[0] += 100
arr = np.arange(5)
c_arr = arr.view(C)
print('before: c_arr = {}'.format(c_arr))
c_arr.my_func()
print('after:  c_arr = {}'.format(c_arr))
print('after:  arr   = {}'.format(arr))

In [ ]:
c_arr.base

* nemohu použít konstruktor z ndarray?

In [ ]:
aux = C([1, 2, 3])
aux

### Dispatch mechanism
* doporučený přístup
* př: CuPy arrays (n-rozměrná pole na GPU)
* užití \_\_array__()

In [ ]:
class Diagonal:
    def __init__(self, N, value):
        self.N = N
        self.value = value
    def __array__(self):
        return self.value * np.eye(5)
arr = Diagonal(5,2)
np.asarray(arr)

In [ ]:
arr = np.multiply(arr, 5)
arr

In [ ]:
type(arr)

## Input & Output
* NumPy má vlastní funkce pro zápis a načítání polí
* komprese
* vytvoří nebo přepíše
* .npy uloží všechny potřebné informace potřebné k rekonstrukci pole (data, dtype, shape)

In [ ]:
arr2d = basic_array()
np.save('src/arr2d',arr2d)
aux = np.load('src/arr2d.npy')
aux

In [ ]:
arr2d += 100
np.save('src/arr2d',arr2d)
aux = np.load('src/arr2d.npy')
aux

* .npz (zip) uloží více polí
 * klíč

In [ ]:
arr = np.array([1,2,3])
np.savez('src/ziparr', arr, y = arr2d)   
aux = np.load('src/ziparr.npz')

aux.files

In [ ]:
aux['arr_0']

In [ ]:
aux['y']

In [ ]:
arr = np.array([1,1])
np.savez('src/ziparr', z = arr)
aux = np.load('src/ziparr.npz')
aux.files

## RNG
* pseudo náhodná čísla
* dříve RandomState
* BitGenerátory vytvoří posloupnosti náhodných čísel
* Generátory transofrmují tyto posloupnosti do posloupností, které se řídí určitým rozdělením
 * Generátory mohou být inicializovány pomocí více BitGenerátorů

In [ ]:
from numpy import random as ran 

In [ ]:
rng = ran.default_rng()     # new instance of Generator
vals1 = rng.standard_normal(10)
vals1

In [ ]:
vals2 = rng.standard_normal(10)
vals2

In [ ]:
vals1 = rng.normal(loc=10,scale=20, size=5)
vals1

  * Generátor obsahuje svůj vlastní interní BitGenerátor

In [ ]:
rng.bit_generator

### Seeding
...zhruba
* chceme reprodukovatelnou posloupnost náhodných čísel
* BitGenerator si vezme libovolně velké přirozené číslo, nebo list takových čísel jako seed
 * problém s kvalitou, tj: chceme kvalitní výstup náhodných čísel bez ohledu na seed
 * delegováno funkci SeedSequence
* krátkodobá paměť

In [ ]:
seed = 123456789
ss = ran.SeedSequence(seed)
rng = ran.default_rng(ss)
vals1 = rng.standard_normal(10)
vals1

In [ ]:
rng = ran.default_rng(ss)
vals2 = rng.standard_normal(10)
vals2

In [ ]:
vals1 - vals2

In [ ]:
rng = ran.default_rng(ss)
vals1 = rng.standard_normal(10)
vals2 = rng.standard_normal(10)
vals1 - vals2

#### Entropie
* atribut SeedSequence
* (berte s rezervou) náhodné číslo, které získá OS. Toto číslo se může například brát z pohybu myši.

In [ ]:
ss = np.random.SeedSequence()
ss.entropy

* obvyklý postup generace náhodných čísel

In [ ]:
ss_aux = np.random.SeedSequence()
ss = np.random.SeedSequence(ss_aux.entropy)
rng = ran.default_rng(ss)
vals = rng.standard_normal(10)
vals

## Lineární algebra
### linalg
* překryv s scipy.linalg
* SciPy obsahuje víc funkcí, např. LU rozklad
* některé funkce se liší v argumentech, např: sc.linalg.eig, np.linalg.solve

In [ ]:
from scipy import linalg

In [ ]:
arr1 = np.array([0,1,2,3,4])
arr2 = np.array([5,6,7,8,9])

In [ ]:
np.inner(arr1, arr2)

In [ ]:
np.outer(arr1, arr2)

### vlastní čísla

In [ ]:
arr2d = np.diag([1,2,3])
np.linalg.eig(arr2d)

In [ ]:
arr2d = np.array([[1, -1], [1, 1]])
np.linalg.eig(arr2d)

* zaokrouhlovací chyby

In [ ]:
arr2d = np.array([[1 + 1e-9, 0], [0, 1 - 1e-9]])
np.linalg.eig(arr2d)

##### SciPy

In [ ]:
arr2d = np.diag([1,2,3])
linalg.eig(arr2d)

* SciPy umožňuje modifikovat matici $\mathbb{B}$ ve vztahu $\mathbb{A} - \lambda\mathbb{B}$
 * defaultně $\mathbb{B} = \mathbb{I}$

In [ ]:
arr2d2 = 2*np.eye(3)
linalg.eig(arr2d, arr2d2)

#### QR algoritmus

In [ ]:
q, r = np.linalg.qr(arr2d)

In [ ]:
q

In [ ]:
r

### LAR

In [ ]:
arr2d = np.diag([1,2,3])
b = [1,1,1]
np.linalg.solve(arr2d, b)

* více najednou

In [ ]:
arr2d2 = np.stack([arr2d, arr2d])
b2 = np.stack([b, b])

In [ ]:
np.linalg.solve(arr2d2, b2)

##### SciPy

In [ ]:
arr2d = np.diag([1,2,3])
b = [1,1,1]
linalg.solve(arr2d, b)

In [ ]:
arr2d2 = np.stack([arr2d, arr2d])
b2 = np.stack([b, b])

* více najednou

In [ ]:
linalg.solve(arr2d2, b2)